There are three crucial variables: inputs (v + II), outputs (r), and the internal variable x

**Inputs → Synaptic + External Inputs**  
The synaptic input \(v\) for neuron \(i\) is defined as

$$
v_i = \sum_{j} W_{ij} r_j
$$

where \(r_j\) is the **output** of neuron \(j\), and \(W_{ij}\) is the weight connecting presynaptic neuron \(j\) to postsynaptic neuron \(i\). Thus, rows correspond to postsynaptic neurons and columns correspond to presynaptic neurons.

**Internal variable → x**  
We update \(x\) according to the differential equations for LIF or QIF neurons. The actual output \(r\) increases by a magnitude \(b\) when \(x\) reaches the threshold, and then decays exponentially.



In [ ]:

def dynamics(x_var, r_var, I_var, nqif, b):
    """
    Compute derivatives of neuron state and adaptation variable.
    
    Inputs:
        x_var   : internal state of neurons
        r_var   : output firing rate or adaptation variable
        I_var   : total input to neurons (external + recurrent)
        nqif    : number of QIF neurons at the start of the array
        b       : adaptation parameter for r
        
    Outputs:
        dx      : derivative of neuron state
        dr      : derivative of adaptation/firing rate
    """
    
    # Initialize dx (derivative of state) as zeros for all neurons
    dx = np.zeros(N)  
    
    # -----------------------------
    # Step 1: Add stochastic noise to inputs
    # -----------------------------
    # LIF neurons: neurons from index nqif onward
    # Generate Gaussian noise for LIF neurons
    I_noise_lif = np.random.randn(N - nqif) * sigman  
    
    # QIF neurons: neurons from index 0 to nqif-1
    # Generate Gaussian noise for QIF neurons
    I_noise_qif = np.random.randn(nqif) * sigman  
    
    # -----------------------------
    # Step 2: Compute derivative for LIF neurons
    # -----------------------------
    # LIF dynamics: dx/dt = -x + I + noise
    # Applies to neurons with indices nqif:N
    dx[nqif:] = -x_var[nqif:] + I_var[nqif:] + I_noise_lif  
    
    # -----------------------------
    # Step 3: Compute derivative for QIF neurons
    # -----------------------------
    # QIF dynamics: dx/dt = 1 - cos(x) + I*(1 + cos(x)) + noise
    # This is a phase-based neuron model, x represents phase in [0, 2π)
    dx[:nqif] = 1 - np.cos(x_var[:nqif]) + I_var[:nqif] * (1 + np.cos(x_var[:nqif])) + I_noise_qif  
    
    # -----------------------------
    # Step 4: Compute derivative for adaptation variable r
    # -----------------------------
    # Adaptation decays exponentially: dr/dt = -b * r
    dr = -b * r_var  
    
    return dx, dr



def detect(x, xnew, rnew, nspike, nqif, b, vt, vrest):
    """
    Detect spikes and apply neuron-specific resets and adaptation updates.
    
    Inputs:
        x       : neuron internal states at previous timestep
        xnew    : neuron internal states after RK2 integration
        rnew    : adaptation after integration
        nspike  : cumulative spike counts
        nqif    : number of QIF neurons
        b       : adaptation increment for each spike
        vt      : threshold for LIF neurons
        vrest   : resting potential for LIF neurons
    
    Outputs:
        xnew    : updated neuron internal states after spike resets
        rnew    : updated adaptation after spike increments
        nspike  : updated spike counts
    """
    
    # -----------------------------
    # Step 1: Detect spikes in LIF neurons
    # -----------------------------
    # LIF neurons are from index nqif to N-1
    # A spike occurs if membrane potential crosses threshold vt from below
    ispike_lif = np.where(x[nqif:] < vt) and np.where(xnew[nqif:] > vt)
    ispike_lif = ispike_lif[0] + nqif  # adjust indices to full neuron array
    
    # If there are LIF spikes detected
    if len(ispike_lif) > 0:
        # Increment adaptation/firing rate variable r by b for each spiking neuron
        rnew[ispike_lif[:]] = rnew[ispike_lif[:]] + b  
        
        # Reset membrane potential to resting value after spike
        xnew[ispike_lif[:]] = vrest  
        
        # Increment spike count for each neuron that spiked
        nspike[ispike_lif[:]] = nspike[ispike_lif[:]] + 1  
    
    # -----------------------------
    # Step 2: Detect spikes in QIF neurons
    # -----------------------------
    # QIF neurons are from index 0 to nqif-1
    # QIF uses a phase variable x in [0, 2π)
    # A spike occurs if phase advances past π (firing point) in one timestep
    
    # Compute distance from current phase to π
    dpi = np.mod(np.pi - np.mod(x, 2 * np.pi), 2 * np.pi)  
    
    # Detect neurons where phase has advanced past π
    ispike_qif = np.where((xnew[:nqif] - x[:nqif]) > 0) and np.where((xnew[:nqif] - x[:nqif] - dpi[:nqif]) > 0)
    
    # If there are QIF spikes detected
    if len(ispike_qif) > 0:
        # Increment adaptation/firing rate variable r by b
        rnew[ispike_qif[:]] = rnew[ispike_qif[:]] + b  
        
        # QIF neurons don’t reset x explicitly here (phase continues)
        # Increment spike count for each neuron that spiked
        nspike[ispike_qif[:]] = nspike[ispike_qif[:]] + 1  
    
    return xnew, rnew, nspike


In [ ]:
def evolution(x, r, Iext, w, nqif, it, dt, iout, nspike, b, vt, vrest):
    """
    Perform one timestep of network evolution for neurons using RK2 integration.
    
    Inputs:
        x       : neuron internal state vector 
        r       : neuron output
        Iext    : external input current (N neurons x T timesteps)
        w       : recurrent weight matrix (N x N)
        nqif    : neuron type indicator (e.g., fraction of QIF neurons)
        it      : current timestep index
        dt      : simulation timestep
        iout    : index of neuron(s) to record output
        nspike  : spike count / history
        b       : neuron parameter (e.g., adaptation)
        vt      : threshold voltage
        vrest   : resting potential
    
    Outputs:
        x       : updated neuron states
        r       : updated firing rates
        nspike  : updated spike counts
        r[iout] : firing rate of neuron(s) of interest
        II      : external input at this timestep
        v       : recurrent input from other neurons
    """

    # -----------------------------
    # Step 1: Compute external input
    # -----------------------------
    # Iext[:, it] selects all neurons at timestep 'it'
    # np.asarray ensures the result is a proper NumPy array
    # np.squeeze removes any singleton dimensions, e.g., shape (N,1) -> (N,)
    II = np.squeeze(np.asarray(Iext[:, it]))  
    
    # -----------------------------
    # Step 2: Compute recurrent input
    # -----------------------------
    # Multiply weight matrix w by firing rates r to get recurrent input as in the equation on the first markdown cell
    # If w is a sparse matrix, .A1 converts it to a 1D NumPy array
    v = w.dot(r.T).A1  
    
    # -----------------------------
    # Step 3: RK2 Integration
    # -----------------------------
    # Compute derivatives at current state
    # dx: derivative of neuron state x
    # dr: derivative of firing rate r
    dx, dr = dynamics(x, r, II + v, nqif, b)  
    
    # Estimate midpoint values for RK2
    xnew = x + dt * dx / 2  
    rnew = r + dt * dr / 2  
    
    # Compute derivatives at midpoint for better accuracy
    dx, dr = dynamics(xnew, rnew, II + v, nqif, b)  
    
    # Update states and firing rates for the next timestep
    xnew = x + dt * dx  
    rnew = r + dt * dr  
    
    # -----------------------------
    # Step 4: Spike detection and reset
    # -----------------------------
    # Detect spikes based on threshold vt and resting potential vrest
    # Update spike counts in nspike
    xnew, rnew, nspike = detect(x, xnew, rnew, nspike, nqif, b, vt, vrest)  
    
    # -----------------------------
    # Step 5: Update state variables
    # -----------------------------
    # Copy updated values to avoid reference issues
    x, r = np.copy(xnew), np.copy(rnew)  

    # -----------------------------
    # Step 6: Return results
    # -----------------------------
    # x, r        : updated neuron states and firing rates
    # nspike      : updated spike counts
    # r[iout]     : firing rate(s) of neuron(s) of interest
    # II          : external input at this timestep
    # v           : recurrent input from network
    return x, r, nspike, r[iout], II, v
